In [10]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import dask.array as da
import os
import os.path as osp
import sgkit_plink 
from dotenv import load_dotenv; load_dotenv();

In [2]:
path = osp.join(os.environ['GWAS_TUTORIAL_DIR'], 'HapMap_3_r3_1')
path

'/home/jovyan/work/data/gwas/tutorial/1_QC_GWAS/HapMap_3_r3_1'

In [88]:
%%time
ds = sgkp.read_plink(path, bim_sep='\t', fam_sep=' ', bim_int_contig=False)
ds

CPU times: user 1.33 s, sys: 127 ms, total: 1.46 s
Wall time: 1.45 s


<xarray.Dataset>
Dimensions:             (alleles: 2, ploidy: 2, samples: 165, variants: 1457897)
Dimensions without coordinates: alleles, ploidy, samples, variants
Data variables:
    variant/contig      (variants) int16 0 0 0 0 0 0 0 ... 23 23 23 23 23 23 23
    variant/position    (variants) int32 dask.array<chunksize=(1457897,), meta=np.ndarray>
    variant/alleles     (variants, alleles) |S1 dask.array<chunksize=(1457897, 1), meta=np.ndarray>
    sample/id           (samples) <U1 dask.array<chunksize=(165,), meta=np.ndarray>
    call/genotype       (variants, samples, ploidy) int8 dask.array<chunksize=(208271, 165, 2), meta=np.ndarray>
    call/genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(208271, 165, 2), meta=np.ndarray>
    variant/id          (variants) <U1 dask.array<chunksize=(1457897,), meta=np.ndarray>
    sample/family_id    (samples) <U1 dask.array<chunksize=(165,), meta=np.ndarray>
    sample/paternal_id  (samples) <U1 dask.array<chunksize=(165,), meta=np.ndarray>
    sample/maternal_id  (samples) <U1 dask.array<chunksize=(165,), meta=np.ndarray>
    sample/sex          (samples) int8 dask.array<chunksize=(165,), meta=np.ndarray>
    sample/phenotype    (samples) int8 dask.array<chunksize=(165,), meta=np.ndarray>
Attributes:
    contigs:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',...

In [89]:
x = ds['call/genotype']

In [157]:
import xarray as xr
from xarray import Dataset
from typing_extensions import Literal

Dimension = Literal['samples', 'variants']

def _swap(dim: Dimension) -> Dimension:
    return 'samples' if dim == 'variants' else 'variants'

def call_rate(ds: Dataset, dim: Dimension) -> Dataset:
    odim = _swap(dim)[:-1]
    n_called = (~ds['call/genotype_mask'].any(dim='ploidy')).sum(dim=dim)
    return xr.Dataset({
        f'{odim}/n_called': n_called,
        f'{odim}/call_rate': n_called / ds.dims[dim]
    })

def genotype_count(ds: Dataset, dim: Dimension) -> Dataset:
    odim = _swap(dim)[:-1]
    mask, gt = ds['call/genotype_mask'].any(dim='ploidy'), ds['call/genotype']
    non_ref = (gt > 0).any(dim='ploidy')
    hom_alt = ((gt > 0) & (gt[..., 0] == gt)).all(dim='ploidy')
    hom_ref = (gt == 0).all(dim='ploidy')
    het = ~(hom_alt | hom_ref)
    agg = lambda x: xr.where(mask, False, x).sum(dim=dim)
    return xr.Dataset({
        f'{odim}/n_het': agg(het),
        f'{odim}/n_hom_ref': agg(hom_ref),
        f'{odim}/n_hom_alt': agg(hom_alt),
        f'{odim}/n_non_ref': agg(non_ref)
    })

def allele_count(ds: Dataset) -> Dataset:
    # Collapse 3D calls into 2D array where calls are flattened into columns
    gt = ds['call/genotype'].stack(calls=('samples', 'ploidy'))
    mask = ds['call/genotype_mask'].stack(calls=('samples', 'ploidy'))
    
    # Count number of non-missing alleles (works with partial calls)
    an = (~mask).sum(dim='calls')
    # Count number of each individual allele 
    ac = xr.concat([
        xr.where(mask, 0, gt == i).sum(dim='calls')
        for i in range(ds.dims['alleles'])
    ], dim='alleles').T
    
    return xr.Dataset({
        'variant/allele_count': ac,
        'variant/allele_total': an,
        'variant/allele_frequency': ac / an
    })

def variant_stats(ds: Dataset) -> Dataset:
    return xr.merge([
        call_rate(ds, dim='samples'),
        genotype_count(ds, dim='samples'),
        allele_count(ds)
    ])

def sample_stats(ds: Dataset) -> Dataset:
    return xr.merge([
        call_rate(ds, dim='variants'),
        genotype_count(ds, dim='variants')
    ])

In [158]:
xr.set_options(display_width=80, display_style='text')

In [159]:
ds = sgkit_plink.read_plink(path, bim_sep='\t', fam_sep=' ', bim_int_contig=False)
xr.merge([variant_stats(ds), sample_stats(ds)]).compute()

<xarray.Dataset>
Dimensions:                   (alleles: 2, samples: 165, variants: 1457897)
Dimensions without coordinates: alleles, samples, variants
Data variables:
    variant/n_called          (variants) int64 165 161 165 165 ... 162 165 164
    variant/call_rate         (variants) float64 1.0 0.9758 1.0 ... 1.0 0.9939
    variant/n_het             (variants) int64 0 0 0 46 37 46 ... 82 86 85 0 45
    variant/n_hom_ref         (variants) int64 165 160 165 116 ... 40 39 165 116
    variant/n_hom_alt         (variants) int64 0 1 0 3 3 3 3 ... 37 21 37 38 0 3
    variant/n_non_ref         (variants) int64 0 1 0 49 40 ... 103 123 123 0 48
    variant/allele_count      (variants, alleles) int64 330 0 320 2 ... 0 277 51
    variant/allele_total      (variants) int64 330 322 330 330 ... 324 330 328
    variant/allele_frequency  (variants, alleles) float64 1.0 0.0 ... 0.1555
    sample/n_called           (samples) int64 1453694 1437110 ... 1455881
    sample/call_rate          (samples) float64 0.9971 0.9857 ... 0.9835 0.9986
    sample/n_het              (samples) int64 399167 384755 ... 391392 397963
    sample/n_hom_ref          (samples) int64 959731 954114 ... 946913 962834
    sample/n_hom_alt          (samples) int64 94796 98241 98518 ... 95568 95084
    sample/n_non_ref          (samples) int64 493963 482996 ... 486960 493047

In [160]:
dss = variant_stats(ds)
dss

<xarray.Dataset>
Dimensions:                   (alleles: 2, variants: 1457897)
Dimensions without coordinates: alleles, variants
Data variables:
    variant/n_called          (variants) int64 dask.array<chunksize=(208271,), meta=np.ndarray>
    variant/call_rate         (variants) float64 dask.array<chunksize=(208271,), meta=np.ndarray>
    variant/n_het             (variants) int64 dask.array<chunksize=(208271,), meta=np.ndarray>
    variant/n_hom_ref         (variants) int64 dask.array<chunksize=(208271,), meta=np.ndarray>
    variant/n_hom_alt         (variants) int64 dask.array<chunksize=(208271,), meta=np.ndarray>
    variant/n_non_ref         (variants) int64 dask.array<chunksize=(208271,), meta=np.ndarray>
    variant/allele_count      (variants, alleles) int64 dask.array<chunksize=(208271, 1), meta=np.ndarray>
    variant/allele_total      (variants) int64 dask.array<chunksize=(208271,), meta=np.ndarray>
    variant/allele_frequency  (variants, alleles) float64 dask.array<chunksize=(208271, 1), meta=np.ndarray>

In [161]:
#ds_ac['variant/allele_frequency'].values[:10]

In [162]:
import pytest
import xarray as xr
import numpy as np
from numpy import ndarray
from sgkit.testing import simulate_genotype_call_dataset
from sgkit.stats.aggregation import allele_count

def get_dataset(calls, **kwargs):
    calls = np.asarray(calls)
    ds = simulate_genotype_call_dataset(
        n_variant=calls.shape[0], n_sample=calls.shape[1], **kwargs
    )
    dims = ds["call/genotype"].dims
    ds["call/genotype"] = xr.DataArray(calls, dims=dims)
    ds["call/genotype_mask"] = xr.DataArray(calls < 0, dims=dims)
    return ds

In [163]:
ds = get_dataset([ 
    [[2, 2], [1, 1], [0, 0]],
    [[0, 1], [1, 2], [2, 1]],
    [[-1, 0], [-1, 1], [-1, 2]],
    [[-1, -1], [-1, -1], [-1, -1]],
])

In [165]:
print(genotype_count(ds, dim='samples').to_dataframe().to_markdown())

|   variants |   variant/n_het |   variant/n_hom_ref |   variant/n_hom_alt |   variant/n_non_ref |
|-----------:|----------------:|--------------------:|--------------------:|--------------------:|
|          0 |               0 |                   1 |                   2 |                   2 |
|          1 |               3 |                   0 |                   0 |                   3 |
|          2 |               0 |                   0 |                   0 |                   0 |
|          3 |               0 |                   0 |                   0 |                   0 |


In [85]:
def apply(ds, fn):
    return ds.merge(fn(ds))
ds.pipe(apply, allele_count)

<xarray.Dataset>
Dimensions:               (alleles: 2, ploidy: 2, samples: 2, variants: 4)
Dimensions without coordinates: alleles, ploidy, samples, variants
Data variables:
    variant/contig        (variants) int64 0 0 0 0
    variant/position      (variants) int64 0 1 2 3
    variant/alleles       (variants, alleles) |S1 b'T' b'C' b'T' ... b'A' b'C'
    sample/id             (samples) <U2 'S0' 'S1'
    call/genotype         (variants, samples, ploidy) int8 1 0 1 0 1 ... 0 0 0 0
    call/genotype_mask    (variants, samples, ploidy) bool False False ... False
    variant/allele_count  (variants, alleles) int64 2 2 1 3 2 2 4 0
Attributes:
    contigs:  [0]